# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
reviews = pd.read_csv("reviews_sample.csv",index_col=0)
recipes = pd.read_csv("recipes_sample.csv")
recipes = recipes [['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]
recipes

recipes_2 = pd.read_csv("recipes_sample.csv", usecols = ['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])
recipes_2

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
reviews1 = reviews.sample(int(0.05*reviews.shape[0]))
recipes1 = recipes.sample(int(0.05*recipes.shape[0]))
print(recipes1.shape)
with pd.ExcelWriter("recipes.xlsx",engine='xlsxwriter') as writer:
    recipes1.to_excel(writer, sheet_name="Рецепты")
    reviews1.to_excel(writer, sheet_name = "Отзывы")

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
file = xw.Book('recipes.xlsx')
n = file.sheets('Рецепты')
n.range('H2:H1501').formula = '=D2*60'
n.range('H1').value = ['seconds_formula']

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
file.sheets[0].range('H1').font.bold = True
n.range('H1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
count=2
for i in n.range('D2:D1501').value:
    if i<5:
        n['D'+str(count)].color = (153, 255, 153)
    elif 5<=i<10:
        n['D'+str(count)].color = (255, 255, 153)
    else:
        n['D'+str(count)].color = (255, 153, 153)
    count+=1

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
n = file.sheets('Рецепты')
n.range('I2').formula = '=COUNTIF(Отзывы!$C$2:$C$6335;B2)'
n.range('I1').value = 'n_reviews'

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
id = np.array(recipes['id'])
count = 2
file.sheets[1].activate()
m =  file.sheets('Отзывы')
for j in n.range('C2:С6334').value:
    if (j not in x) or m['E'+count]<0 or m['E'+count]>5:
        m['A'+count].color = (255,51,51) 
    count+=1

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
with open('recipes_model.csv',encoding="utf-8") as csvfile:
    f = csv.reader(csvfile, delimiter='	')
    b = list(f)
print(b)
        
new = file.sheets.add("Модель")
new.range('A2').value = b

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
x =  file.sheets('Модель')
x["A2:G2"].font.color = "#00ccff"
x.range("A2:I18").columns.autofit()
x.range("A2:G2").font.bold = True
x.show_autofilter = True

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 